## Descrição do desafio

O problema proposto usa como base quatro datasets, cada um contendo dados de empresas abrangendo todos os tipos e setores. O primeiro dataset contem os dados de todas as empresas a serem analisadas; os outros três são subsets do primeiro e se referem ao portifolio de empresas que prestam serviços as empresas dos respectivos portifolios.

O objetivo do desafio é criar um modelo que faça recomendações de possíveis leads para as empresas que prestam serviço baseado em seus portifolios.

## Imports

In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
import category_encoders as ce
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [2]:
# Import do dataset geral
EM = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\datasets\\estaticos_market.csv')

# Import dos Dados de Portifolio
P1 = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\datasets\\estaticos_portfolio1.csv')
P2 = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\datasets\\estaticos_portfolio2.csv')
P3 = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\datasets\\estaticos_portfolio3.csv')

## Análise Exploratória

In [3]:
EM.head()

,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,...,43.738462,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0
1,1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
P1.head()

,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,0.646575,<= 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...,False,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,6.586301,5 a 10,...,NaN,NaN,NaN,0.0,0.0,2.0,NaN,-100.0,0.0,4
2,2,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...,True,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,8.010959,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,COMERCIO VAREJISTA,COMERCIO,20.863014,> 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,18.726027,15 a 20,...,39.454762,91.2,19.333333,14.0,15.0,25.0,-6.666667,-44.0,0.0,3


In [5]:
P2.head()

,Unnamed: 0,id
0,0,09e95c1a84049001d086470a8f320a19b076f955a89122...
1,1,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...
2,2,16843c9ffb920170477118933798510d8d5f00f5c56c12...
3,3,ff045934d90aab184dd15e66daf3b3c97193bda5449298...
4,4,a0ce528caab2c62497a548d0ebd9e935143ef81ed7dbf0...


In [6]:
P3.head()

,Unnamed: 0,id
0,0,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...
1,1,16843c9ffb920170477118933798510d8d5f00f5c56c12...
2,2,1b8d092daff12bed8b3c2ab93bf5df2921865a68aca185...
3,3,3aafa3a1f12f483740a52956af100223333d0e3c79bf2c...
4,4,c60431fbca90067f105570e1e8a684bcd7048205bbf6c4...


In [7]:
EM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462298 entries, 0 to 462297
Columns: 182 entries, Unnamed: 0 to qt_filiais
dtypes: bool(9), float64(144), int64(2), object(27)
memory usage: 614.1+ MB


In [8]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
EM.dtypes

Unnamed: 0                                         int64
id                                                object
fl_matriz                                           bool
de_natureza_juridica                              object
sg_uf                                             object
natureza_juridica_macro                           object
de_ramo                                           object
setor                                             object
idade_empresa_anos                               float64
idade_emp_cat                                     object
fl_me                                               bool
fl_sa                                               bool
fl_epp                                              bool
fl_mei                                              bool
fl_ltda                                             bool
dt_situacao                                       object
fl_st_especial                                      bool
fl_email                       

In [9]:
# Frequencia de valores faltantes nas colunas
Nan_Ratio = pd.DataFrame((EM.isna().sum())/(len(EM))).sort_values(by=0,ascending=False)
Nan_Ratio = Nan_Ratio.rename(columns={0:'NaN Ratio'})
Nan_Ratio

,NaN Ratio
grau_instrucao_macro_desconhecido,1.000000
qt_alteracao_socio_180d,1.000000
qt_alteracao_socio_365d,1.000000
qt_alteracao_socio_total,1.000000
qt_alteracao_socio_90d,1.000000
idade_media_coligadas_baixadas,0.999706
coligada_mais_antiga_baixada,0.999706
coligada_mais_nova_baixada,0.999706
vl_total_veiculos_antt,0.999619
vl_total_tancagem,0.999394


In [10]:
EM.select_dtypes(exclude=['int64','float64']).describe()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
count,462298,462298,462298,462298,462298,462298,460371,462298,462298,462298,462298,462298,462298,462292,462298,462298,462298,462298,460371,460371,460371,460371,460371,379585,379585,460359,447447,447447,451130,43893,460371,403600,403600,460371,434785,434785
unique,462298,2,67,6,7,33,5,6,2,2,1,2,2,7334,2,2,2,2,87,21,2,2,2,2,2,27,6,5,4,2,2,19,73,2,12,11
top,a6b5036e8e016e6ee424096fa8657f6a83f7d3c11be22f...,True,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,1 a 5,False,False,False,False,False,2005-11-03,False,False,True,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,False,False,False,True,False,MA,VERDE,ACIMA DE 1 ANO,MEDIA,BOA,False,CENTRO AMAZONENSE,MANAUS,True,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00"
freq,1,433232,295756,127654,320211,172404,211224,138580,461083,453866,462298,311398,461056,69465,462230,256228,335468,236779,172404,211224,460091,457095,429687,199617,285545,124823,145430,378896,217949,43879,460030,71469,60008,264741,273861,252602


In [11]:
EM.describe()

,Unnamed: 0,idade_empresa_anos,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,nu_meses_rescencia,vl_frota,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
count,462298.00000,462298.000000,280.000000,176.000000,30684.000000,30684.000000,6590.000000,460371.000000,460371.000000,1760.000000,336.000000,3683.000000,417022.000000,2.419000e+04,318469.000000,347207.000000,347207.000000,347207.000000,310696.000000,310696.000000,310696.000000,307381.000000,3251.000000,195115.000000,145029.000000,6743.000000,0.0,0.0,0.0,0.0,4544.000000,4544.000000,4544.000000,4544.000000,4544.000000,4544.000000,4544.000000,4544.000000,65391.000000,65391.000000,65391.000000,65391.000000,65391.000000,65391.000000,65391.000000,65391.0,65391.000000,65391.000000,65391.000000,65147.000000,65147.000000,65147.000000,136.000000,136.000000,136.000000,65391.

In [12]:
# Filtrando IDs dos portifolios no dataset principal
EM_P1 = EM[EM['id'].isin(list(P1['id']))]
EM_P2 = EM[EM['id'].isin(list(P2['id']))]
EM_P3 = EM[EM['id'].isin(list(P3['id']))]

In [13]:
EM_P1.select_dtypes(exclude=['int64','float64']).describe()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
count,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,454,454,555,544,544,551,54,555,491,491,555,530,530
unique,555,2,6,6,2,2,3,6,2,2,1,2,1,416,1,2,2,2,8,2,1,2,2,2,2,7,6,3,4,1,1,19,66,2,7,9
top,a450a3cdacf9b9c26f1edd55535e3a87d9449c09ee3d57...,True,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,1 a 5,False,False,False,False,False,2005-11-03,False,False,True,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,False,False,False,True,False,MA,VERDE,ACIMA DE 1 ANO,MEDIA,BOA,False,CENTRO AMAZONENSE,MANAUS,False,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00"
freq,1,519,432,157,447,496,496,157,553,546,555,337,555,100,555,300,400,311,496,496,555,551,517,286,316,156,167,453,280,54,555,78,65,442,325,296


In [14]:
EM_P1.describe()

,Unnamed: 0,idade_empresa_anos,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,nu_meses_rescencia,vl_frota,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
count,555.000000,555.000000,0.0,0.0,38.000000,38.000000,4.000000,555.000000,555.000000,0.0,0.0,3.0,508.000000,31.000000,405.000000,408.000000,408.000000,408.000000,362.000000,362.000000,362.000000,360.000000,7.0,198.000000,190.000000,4.00,0.0,0.0,0.0,0.0,2.000000,2.0,2.0,2.0,2.0,2.000000,2.000000,2.000000,63.000000,63.000000,63.000000,63.000000,63.0,63.000000,63.0,63.0,63.000000,63.000000,63.000000,62.000000,62.000000,62.000000,0.0,0.0,0.0,63.000000,63.0,63.0,63.000000,63.0,63.000000,63.0,63.000000,63.000000,63.000000,63.0,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,46.000000,46.000000,46.000000,46.000000,46.000000,3.700000e+01,3.7

In [15]:
EM_P2.select_dtypes(exclude=['int64','float64']).describe()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
count,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,566,441,566,496,496,566,566,566
unique,566,1,26,6,4,29,5,3,2,2,1,1,2,207,2,2,2,2,60,19,2,2,2,2,1,6,5,1,3,1,2,18,60,2,6,6
top,67f774985fe36dd9a74f5893f2153739404a3d5b6e9643...,True,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,> 20,False,False,False,False,False,2005-11-03,False,False,True,SIM,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,False,False,True,False,False,AM,VERDE,ACIMA DE 1 ANO,ALTA,BOA,False,CENTRO AMAZONENSE,MANAUS,True,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00"
freq,1,566,287,142,323,163,336,379,564,543,566,566,562,201,564,304,307,307,163,163,539,474,490,539,566,142,376,566,523,441,565,138,126,490,278,247


In [16]:
EM_P2.describe()

,Unnamed: 0,idade_empresa_anos,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,nu_meses_rescencia,vl_frota,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
count,566.000000,566.000000,27.000000,5.00000,490.000000,490.000000,262.000000,566.000000,566.000000,30.000000,5.00000,94.000000,566.000000,4.680000e+02,496.000000,382.000000,382.000000,382.000000,353.000000,353.000000,353.000000,352.000000,1.0,310.000000,197.000000,32.000000,0.0,0.0,0.0,0.0,51.000000,51.0,51.000000,51.0,51.0,51.000000,51.000000,51.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,299.0,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,1.0,1.0,1.0,299.000000,299.0,299.000000,299.000000,299.0,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00

In [17]:
EM_P3.select_dtypes(exclude=['int64','float64']).describe()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,fl_optante_simples,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
count,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,265,263,263,265,265,265,265,194,265,241,241,265,265,265
unique,265,1,25,6,4,16,1,6,1,2,1,1,2,129,1,2,2,2,26,13,2,2,2,2,1,6,5,3,3,1,2,19,56,1,6,6
top,30a9521eb087b73e83542c3f30c85cc53e44fb18d60fcf...,True,MUNICIPIO,MA,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,> 20,False,False,False,False,False,2005-11-03,False,False,False,NAO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,False,False,True,False,False,MA,VERDE,ACIMA DE 1 ANO,ALTA,BOA,False,NORTE MARANHENSE,MANAUS,True,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00"
freq,1,265,108,74,128,150,265,176,265,260,265,265,264,85,265,147,133,145,150,150,255,253,221,252,263,74,159,263,236,194,264,41,38,265,138,135


In [18]:
EM_P3.describe()

,Unnamed: 0,idade_empresa_anos,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,nu_meses_rescencia,vl_frota,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
count,265.000000,265.000000,10.00000,0.0,221.000000,221.000000,130.000000,265.000000,265.000000,10.000000,0.0,22.000000,264.000000,2.080000e+02,217.000000,98.000000,98.000000,98.000000,93.000000,93.000000,93.000000,93.000000,0.0,79.000000,43.000000,12.000000,0.0,0.0,0.0,0.0,15.000000,15.0,15.0,15.0,15.0,15.000000,15.000000,15.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.0,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,1.0,1.0,1.0,76.000000,76.0,76.000000,76.000000,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,63.000000,63.000000,63.

## Data pre-processing

### Preenchendo valores faltantes

In [19]:
EM = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\datasets\\estaticos_market.csv')
Nan_Ratio = pd.DataFrame((EM.isna().sum())/(len(EM))).sort_values(by=0,ascending=False)
Nan_Ratio = Nan_Ratio.rename(columns={0:'NaN Ratio'})

# Filtrando colunas com menos de 30% de valoress faltantes
colunas = list(Nan_Ratio[Nan_Ratio['NaN Ratio']<0.3].index)

# Coluna dt_situação, Unnamed: 0, nm_meso_regiao e nm_micro_regiao foram dropadas
del colunas[26:29]
del colunas[5:7]
colunas = sorted(colunas)
colunas.insert(0,'id')
EM = EM[colunas]

# Linhas com valores faltantes na coluna nm_divisao foram dropados
EM = EM.dropna(subset = ['nm_divisao'])
EM = EM.reset_index(drop=True)

# Coluna fl_rm transformada para binário
EM['fl_rm'] = EM['fl_rm'].map({'SIM':True,'NAO':False})

# Valores faltantes nas colunas binárias foram inputados como False 
EM[colunas[8:25]] = EM[colunas[8:25]].fillna(False)

# Filtrando colunas numéricas
colunas_num = list(EM.select_dtypes(include=['int64','float64']).columns)

# Substituindo valores faltantes
EM[colunas_num[1]] = EM[colunas_num[1]].fillna(0)
EM[colunas_num[3:5]] = EM[colunas_num[3:5]].fillna(1)
EM[colunas_num[5]] = EM[colunas_num[5]].fillna(0)
EM[colunas_num[6:8]] = EM[colunas_num[6:8]].fillna(EM[colunas_num[6:8]].median())
EM[colunas_num[8:10]] = EM[colunas_num[8:10]].fillna(0)

# Filtrando colunas object
colunas_obj = list(EM.select_dtypes(include=['object']).columns)
colunas_obj.remove('id')

# Substituindo valores faltantes
EM[colunas_obj[0:2]] = EM[colunas_obj[0:2]].fillna('DE R$ 81.000,01 A R$ 360.000,00')
EM[colunas_obj[3]] = EM[colunas_obj[3]].fillna('MEDIA')
EM[colunas_obj[5]] = EM[colunas_obj[5]].fillna('SEM INFORMACAO')
EM[colunas_obj[6]] = EM[colunas_obj[6]].fillna('CINZA')
EM[colunas_obj[13]] = EM[colunas_obj[13]].fillna(EM[colunas_obj[12]])
EM.to_csv('EM.csv', index=False)

### Encodando e Padronizando

In [ ]:
# Criando dataset para encoding
EM_enc = EM.drop('id',axis=1)

# Hashing Encoder
HE = ce.HashingEncoder(n_components=30, cols=EM_enc.columns)
EM_enc = HE.fit_transform(EM_enc)

# Padronizando os dados
scaler = StandardScaler()
EM_enc = scaler.fit_transform(EM_enc)
EM_enc = pd.DataFrame(EM_enc)

# Exportando dados
EM_enc.to_csv('EM_enc.csv', index=False)

## Modelo

### Validação

In [ ]:
EM_enc = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\EM_enc.csv')
EM = pd.read_csv('C:\\Users\\Luiz Guilherme\\codenation\\projeto_final\\EM.csv')
y_P1 = EM['id'].isin(list(P1['id']))
y_P2 = EM['id'].isin(list(P2['id']))
y_P3 = EM['id'].isin(list(P3['id']))

In [ ]:
# Dividindo dados em treino e teste
X_train_P1, X_test_P1, y_train_P1, y_test_P1 = train_test_split(EM_enc, y_P1, test_size=0.3, shuffle=False)
X_train_P2, X_test_P2, y_train_P2, y_test_P2 = train_test_split(EM_enc, y_P2, test_size=0.3, shuffle=False)
X_train_P3, X_test_P3, y_train_P3, y_test_P3 = train_test_split(EM_enc, y_P3, test_size=0.3, shuffle=False)

# Oversampling
os = SMOTE(random_state=0)
X_train_P1,y_train_P1=os.fit_sample(X_train_P1, y_train_P1)
X_train_P2,y_train_P2=os.fit_sample(X_train_P2, y_train_P2)
X_train_P3,y_train_P3=os.fit_sample(X_train_P3, y_train_P3)

In [ ]:
# Validação cruzada com regressão logística
lrcv = LogisticRegressionCV(cv=5,
                            scoring='balanced_accuracy',
                            solver='saga',
                            n_jobs=-1,
                            refit=True,
                            multi_class='ovr')

In [ ]:
# Obtendo scores para cada protifolio (balanced accuracy)
lr_P1 = lrcv.fit(X_train_P1, y_train_P1)
lr_P1.score(X_train_P1, y_train_P1)

In [ ]:
lr_P2 = lrcv.fit(X_train_P2, y_train_P2)
lr_P2.score(X_train_P2, y_train_P2)

In [ ]:
lr_P3 = lrcv.fit(X_train_P3, y_train_P3)
lr_P3.score(X_train_P3, y_train_P3)

### Performance nos dados de teste

In [ ]:
# Função que plota curva ROC
def ROC(X_train,X_test,y_train,y_test):
    lr=LogisticRegression(solver='saga',
                          n_jobs=-1,
                          multi_class='ovr')
    lr.fit(X_train, y_train)
    lr_roc = roc_auc_score(y_test, lr.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % lr_roc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()

In [ ]:
# Plotando curva ROC para portifolio 1
ROC(X_train_P1,X_test_P1,y_train_P1,y_test_P1)

In [ ]:
# Plotando curva ROC para portifolio 2
ROC(X_train_P2,X_test_P2,y_train_P2,y_test_P2)

In [ ]:
# Plotando curva ROC para portifolio 3
ROC(X_train_P3,X_test_P3,y_train_P3,y_test_P3)

## Obtendo Leads

In [ ]:
# Função que retorna os indexes dos leads no dataset
def Leads(y_P,n_leads):
    lr=LogisticRegression(solver='saga',
                          n_jobs=-1,
                          multi_class='ovr')
    lr.fit(EM_enc, y_P)
    proba = pd.DataFrame(lr.predict_proba(EM_enc)[:,1])
    proba = proba.drop(list(y_P[y_P==True].index))
    leads = list(proba.sort_values(0,ascending=False)[0:n_leads].index)
    return EM.loc[leads]

In [ ]:
# Retornando 1000 leads para cada portifolio
leads_P1 = Leads(y_P1,1000)
leads_P2 = Leads(y_P2,1000)
leads_P3 = Leads(y_P3,1000)

In [ ]:
leads_P1.head()

In [ ]:
leads_P1.describe()

In [ ]:
EM.loc[list(P1.index)].describe()

In [ ]:
leads_P2.head()

In [ ]:
leads_P2.describe()

In [ ]:
EM.loc[list(P2.index)].describe()

In [ ]:
leads_P3.head()

In [ ]:
leads_P3.describe()

In [ ]:
EM.loc[list(P3.index)].describe()